In [2]:
from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import math

In [3]:
df_train=pd.read_csv('data/data_train_with_features.csv')
df_train.head()

,Unnamed: 0,classification,text,sentences,num_sentences,words,num_words,lemmas,stops,num_stops
0,0,Scientific,the interest in anchoring phenomena and phenom...,['the interest in anchoring phenomena and phen...,222,"['interest', 'anchoring', 'phenomena', 'phenom...",3533,"['interest', 'anchor', 'phenomenon', 'phenomen...","[the, in, and, in, has, been, by, their, in, a...",2426
1,1,Scientific,"galaxy clusters , as the largest peaks in the ...","['galaxy clusters , as the largest peaks in th...",372,"['galaxy', 'clusters', 'largest', 'peaks', 'co...",8982,"['galaxy', 'cluster', 'large', 'peak', 'cosmic...","[as, the, in, the, an, in, and, in, for, and, ...",5768
2,2,Scientific,quantum correlations between components of a s...,['quantum correlations between components of a...,138,"['quantum', 'correlations', 'components', 'sys...",2451,"['quantum', 'correlation', 'component', 'syste...","[between, of, a, or, are, on, the, of, whole, ...",1780
3,3,Scientific,methanol masers are often found in star - form...,['methanol masers are often found in star - fo...,141,"['methanol', 'masers', 'found', 'star', 'formi...",3232,"['methanol', 'maser', 'find', 'star', 'form', ...","[are, often, in, are, two, of, to, i, most, th...",1940
4,4,Scientific,interdisciplinary research has recently gained...,['interdisciplinary research has recently gain...,88,"['interdisciplinary', 'research', 'recently', ...",2695,"['interdisciplinary', 'research', 'recently', ...","[has, a, in, the, of, to, the, in, was, to, tw...",1997


In [4]:
def token_count(text):
    tokens = text.split()
    return len(tokens)

In [34]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer,AutoTokenizer,AutoModelForSeq2SeqLM
import torch
#model_name='facebook/bart-large-cnn'
model_name='google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [35]:
import math

def adjust_length(text):
    length = token_count(text)
    if length < 80:
        min_length = length + int(length * 0.05)
        max_length = min_length + 100
    elif length < 100:
        min_length = length + int(length * 0.1)
        max_length = min_length + 100
    else:
        min_length = math.ceil(length / 50) * 50
        max_length = min_length + 100
    return min_length, max_length

In [36]:
def batch_sent(sentenc,splitt=180,split='. '):
    sentences = sentenc.split(split)
    # Erstellen Sie Batches von Sätzen, die weniger als 1024 Tokens enthalten
    batches = []
    batch = []
    batch_len = 0
    for sentence in sentences:
        
        sentence_len = len(tokenizer.tokenize(sentence))
        if sentence_len + batch_len > splitt:
            if sentence_len < splitt:  # überspringen Sie Sätze, die länger als 256 Tokens sind
                batches.append(batch)
                batch = [sentence]
                batch_len = sentence_len
            # wenn ein Satz alleine 1024 Tokens überschreitet, wird er übersprungen
        else:
            batch.append(sentence)
            batch_len += sentence_len
    batches.append(batch)
    return batches

In [73]:
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm
import random
from transformers import pipeline
#summarizer = pipeline("text2text-generation", model="facebook/bart-large-cnn")
#summarizer = pipeline("text2text-generation", model="facebook/bart-base")
summarizer = pipeline("text2text-generation", model="t5-base")



In [ ]:
df_summary_testing = pd.DataFrame(columns=['Min_Kompressionsrate', 'Max_Kompressionsrate', 'Endgueltige_Kompressionsrate', 'Zusammenfassung'])

In [64]:
df_s=df_train.loc[[500, 1000, 1500]]['text'].reset_index(drop=True)

In [50]:
df_s.head()

0    The Administration of Union Territory Daman an...
1    This sound track was beautiful! It paints the ...
Name: text, dtype: object

In [42]:
df_s

0    The Administration of Union Territory Daman an...
1    This sound track was beautiful! It paints the ...
2    PART I\n\nIf you don't like Christmas stories,...
Name: text, dtype: object

In [83]:
counter = 0
komp=1
# Teilen Sie den Text in Sätze --> für tatsächliche umsetzubng
soplitting=True
for text in tqdm(df_s):
    text_gesamt_list=[]
    if soplitting:
        split=' .'
    else:
        split='. '
    for batch in tqdm(batch_sent(text,split=split), desc='Verarbeite Batches'):
        # Zusammenfügen der Sätze in einem Batch
        batch_text = '. '.join(batch)
        min_length_test, max_length_test = adjust_length(batch_text)
        ext_summary=summarizer(batch_text, max_length=int(round(max_length_test*komp,0)), min_length=int(round(min_length_test*komp,0)),length_penalty=100,num_beams=2)
        # Erstellen Sie einen DataFrame für die aktuellen Ergebnisse
        text_gesamt_list.append(ext_summary[0]['generated_text'])
    text_gesamt = '. '.join(text_gesamt_list)
    actual_compression_rate = len(text_gesamt.split(' '))/len(text.split(' '))*100
    df_current = pd.DataFrame({
        'Min_Kompressionsrate': [min_length_test],
        'Max_Kompressionsrate': [max_length_test],
        'Endgueltige_Kompressionsrate': [actual_compression_rate],
        'Zusammenfassung': [text_gesamt]
      
    })

    # Fügen Sie die Daten zum DataFrame hinzu
    df_summary_testing = pd.concat([df_summary_testing, df_current], ignore_index=True)
    counter += 1
    soplitting=False

    # Wenn der Zähler ein Vielfaches von 50 ist, speichern Sie die Daten in einer CSV-Datei
    # if counter % 50 == 0:
    #     df_summary_testing.to_csv(f'/content/df_summary_testing_{counter}.csv', index=False)


100%|██████████| 3/3 [06:21<00:00, 127.00s/it]


In [85]:
df_summary_testing

,Min_Kompressionsrate,Max_Kompressionsrate,Endgueltige_Kompressionsrate,Zusammenfassung
0,63,163,100.000000,The Administration of Union Territory Daman an...
1,78,178,100.000000,This sound track was beautiful! It paints the ...
2,63,163,100.000000,The Administration of Union Territory Daman an...
3,78,178,100.000000,This sound track was beautiful! It paints the ...
4,63,163,100.000000,The Administration of Union Territory Daman an...
5,78,178,100.000000,This sound track was beautiful! It paints the ...
6,107,207,105.567970,And one or two other oldish persons. But they ...
7,63,163,148.333333,its order that made it compulsory for women to...
8,78,178,60.000000,vid. game music! vid. game music! This sound t...
9,107,207,107.635009,the Twilight of Christmas Eve. the Twilight of...


In [78]:
df_s[0]

'The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.'

In [80]:
df_summary_testing[7:8]['Zusammenfassung'][7]

'its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7 . The administration was forced to withdraw the decision within 24 hours after receiving flak from employees and was slammed on social media . Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision after receiving fla   '

In [59]:
df_s
len(df_s[0].split(' '))

17179

In [76]:
len(df_summary_testing['Zusammenfassung'][4].split(' '))

15104

In [77]:
15104/17179*100

87.9212992607253

In [35]:
text=df_s[0]

In [40]:
batch_sent(text,split=' .')[0]

['galaxy clusters , as the largest peaks in the cosmic density field , play an important role in astrophysics and cosmology ( e.g. * ? ? ?* ; * ? ? ?structure formation theory , realized in large scale n - body simulations , makes robust predictions for cluster space density and clustering within various cosmological models',
 ' since clusters are also the most observationally accessible features of large scale structure , they provide an opportunity to place strong constraints on both cosmological parameters and the growth of structure',
 'the great challenge of cluster cosmology lies in confidently relating the dark matter halos we can robustly predict to the baryonic structures we observe',
 'substantial work is being done to close the gap between theory and observations from both sides',
 'on the theory side , numerical simulations of ever increasing complexity and resolution provide new insights into the evolution of baryons within clusters , and to elucidate the connection betwee

In [41]:
batch_text = '. '.join(batch_sent(text,split=' .')[0])

In [42]:
min_length_test, max_length_test = adjust_length(batch_text)

In [43]:
min_length_test

250

In [ ]:
summarizer(batch_text, max_length=max_length_test, min_length=min_length_test,length_penalty=100,num_beams=2)

In [66]:
import nltk
import math

def check_compression_rate(original_text, filtered_text, compression_rate):
    # Tokenize the texts
    original_tokens = nltk.word_tokenize(original_text)
    filtered_tokens = nltk.word_tokenize(filtered_text)

    # Calculate the expected number of tokens in the filtered text
    expected_tokens = len(original_tokens) * compression_rate

    # Calculate the acceptable range of token counts
    min_tokens = expected_tokens * 0.95
    max_tokens = expected_tokens * 1.05

    # Check if the number of tokens in the filtered text is within the acceptable range
    if min_tokens <= len(filtered_tokens) <= max_tokens:
        return True
    else:
        return math.ceil(min_tokens), math.floor(max_tokens)  # round up for min and down for max


In [63]:
result = check_compression_rate(original_text, filtered_text, compression_rate)

if result == True:
    print("The filtered text is within the acceptable compression range.")
else:
    min_tokens, max_tokens = result
    print(f"The filtered text is not within the acceptable compression range. It should have between {min_tokens} and {max_tokens} tokens.")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_35943/3855289726.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/k3/z_svrdgd6sb9lc9bqdfzp_k00000gn/T/ipykernel_35943/3855289726.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'original_text' is not defined

In [67]:
for text in df_s:
    result =check_compression_rate(text, text, 0.8)
    if result == True:
        print("The filtered text is within the acceptable compression range.")
    else:
        min_tokens, max_tokens = result
        print(f"The filtered text is not within the acceptable compression range. It should have between {min_tokens} and {max_tokens} tokens.")

The filtered text is not within the acceptable compression range. It should have between 13497 and 14917 tokens.
The filtered text is not within the acceptable compression range. It should have between 48 and 52 tokens.
The filtered text is not within the acceptable compression range. It should have between 62 and 68 tokens.
The filtered text is not within the acceptable compression range. It should have between 5371 and 5936 tokens.
